In [1]:
import os
import omegaconf
from shell.utils.experiment_utils import *

In [3]:
save_root_dir = "results"
dataset = "mnist"
algo = "monolithic"
seed = 0

In [4]:
experiment = os.path.join(save_root_dir, f"{dataset}_{algo}", dataset,algo, f"seed_{seed}")

In [5]:
config_path = os.path.join(experiment, "hydra_out", ".hydra", "config.yaml")
# read the config file
cfg = omegaconf.OmegaConf.load(config_path)
cfg

{'train': {'component_update_freq': 100, 'num_epochs': 100, 'save_freq': 1}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 10, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': False}, 'net': {'name': 'mlp', 'depth': 4, 'num_init_tasks': 4, 'layer_size': 64, 'dropout': 0.5, 'freeze_encoder': True}, 'sharing_strategy': {'name': 'no_sharing'}, 'seed': 0, 'algo': 'monolithic', 'job_name': 'mnist_monolithic', 'num_agents': 1, 'root_save_dir': 'results', 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 64, 'improvement_threshold': 0.05}}

In [6]:
graph, datasets, NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg = setup_experiment(cfg)

{'train': {'component_update_freq': 100, 'num_epochs': 100, 'save_freq': 1}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 10, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': False}, 'net': {'name': 'mlp', 'depth': 4, 'num_init_tasks': 4, 'layer_size': 64, 'dropout': 0.5, 'freeze_encoder': True}, 'sharing_strategy': {'name': 'no_sharing'}, 'seed': 0, 'algo': 'monolithic', 'job_name': 'mnist_monolithic', 'num_agents': 1, 'root_save_dir': 'results', 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 64, 'improvement_threshold': 0.05}}
i_size 28
num_classes 10
net_cfg {'name': 'mlp', 'depth': 4, 'num_init_tasks': 4, 'layer_size': 64, 'dropout': 0.5, 'freeze_encoder': True, 'i_size': 28, 'num_classes': 10, 'num_tasks': 10}
<class 'shell.learners.er_nocomponents.NoComponentsER'>


In [7]:
net = NetCls(**net_cfg)
net.state_dict()['components.0.weight']

tensor([[ 0.0373,  0.1091,  0.0589,  ...,  0.0183,  0.0088, -0.0387],
        [-0.0087, -0.1061, -0.0293,  ...,  0.0154,  0.0667, -0.0594],
        [-0.0995, -0.0213,  0.0258,  ..., -0.0098, -0.0758,  0.0072],
        ...,
        [-0.1171, -0.0655, -0.0226,  ..., -0.0482,  0.1031,  0.0465],
        [ 0.0452, -0.0565,  0.0103,  ..., -0.0416, -0.1008, -0.1058],
        [ 0.0342, -0.0968, -0.0641,  ..., -0.0409, -0.0496, -0.0465]],
       device='cuda:0')

In [8]:
# path
# 'results/mnist_monolithic/mnist/monolithic/seed_0/agent_0/task_3/checkpoint.pt'

In [18]:
task_id = 3
agent_id = 0
net = load_net(cfg, NetCls, net_cfg, agent_id=agent_id, task_id=task_id)
net.state_dict()['components.0.weight']

tensor([[-0.0229, -0.0347, -0.0285,  ...,  0.1264, -0.0227,  0.0851],
        [ 0.1163,  0.0259,  0.1773,  ..., -0.0458,  0.1439, -0.2391],
        [-0.2512, -0.1421,  0.1690,  ...,  0.1824, -0.2039, -0.0132],
        ...,
        [-0.0140,  0.0317,  0.1261,  ..., -0.1434,  0.3604,  0.0081],
        [ 0.0494, -0.1926, -0.0939,  ..., -0.0503, -0.0507, -0.0950],
        [ 0.0632,  0.0023, -0.0846,  ..., -0.1044, -0.0306, -0.0698]],
       device='cuda:0')

In [19]:
dataset = datasets[agent_id]
task_id = 9
testloaders = {task: torch.utils.data.DataLoader(testset,
                                                         batch_size=128,
                                                         shuffle=False,
                                                         num_workers=0,
                                                         pin_memory=True,
                                                         ) for task, testset in enumerate(dataset.testset[:(task_id+1)])}
testloaders

{0: <torch.utils.data.dataloader.DataLoader at 0x7fddc0f920e0>,
 1: <torch.utils.data.dataloader.DataLoader at 0x7fddbdf15150>,
 2: <torch.utils.data.dataloader.DataLoader at 0x7fddbdf47be0>,
 3: <torch.utils.data.dataloader.DataLoader at 0x7fddbdf47c40>,
 4: <torch.utils.data.dataloader.DataLoader at 0x7fddbdf479a0>,
 5: <torch.utils.data.dataloader.DataLoader at 0x7fddbdf47a90>,
 6: <torch.utils.data.dataloader.DataLoader at 0x7fddbdf45f30>,
 7: <torch.utils.data.dataloader.DataLoader at 0x7fddbdf45db0>,
 8: <torch.utils.data.dataloader.DataLoader at 0x7fddbdf45ea0>,
 9: <torch.utils.data.dataloader.DataLoader at 0x7fddbdf457e0>}

In [20]:
eval_net(net, testloaders)

{0: 0.9497863247863247,
 1: 0.946285140562249,
 2: 0.9680412371134021,
 3: 0.9698795180722891,
 4: 0.0,
 5: 0.3210401891252955,
 6: 0.3308784850078906,
 7: 0.0,
 8: 0.42332830568124685,
 9: 0.18693467336683417}

In [24]:
dataset.class_sequence

array([5, 0, 4, 3, 4, 6, 4, 3, 8, 0, 1, 0, 5, 9, 6, 4, 9, 0, 3, 0])

In [23]:
for testloader in testloaders.values():
    print(eval_net_task(net, 3, testloader))


0.0
0.9698795180722891
0.4850515463917526
0.9698795180722891
0.0
0.0
0.0
0.4850515463917526
0.0
0.4979899497487437
